深入NumPy库的数值计算，包含ndarray类型（type）更多的细节描述和更高级的数组操作和算法。

**NumPymarkdown**

NumPy (short for Numerical Python)是高性能科学计算和数据分析的基础包。其部分功能如下：
+ ndarray，一个提供快速面向数组算术运算和复杂广播功能的高效多维数组
+ 数学函数——可以对整个数组进行快速运算，而不用写循环
+ 工具——读写磁盘数据以及操作内存映射文件
+ 线性代数、随机数生成和傅里叶变换
+ C API——用于将NumPy与用C、C++或者FORTRAN编写的库连接

NumPy在Python中对数值计算如此重要的原因之一是designed for efficiency on large arrays of data，原因如下：
+ NumPy internally stores data in a contiguous block of memeory, independent of other built-in Python objects. NumPy's library of algorithms written in the C language can operate on this memory without any type checking or other overhead.
+ NumPy arrays also use much less memory than built-in Python sequences.与Python序列相比，占用更少的内存。
+ Numpy operations perform complex computations on entire arrays without the need for Python for loops.对整个数组进行操作而不用写循环。

# 1 ndarray对象的内部机制（ndarray object internals）
NumPy的ndarray提供了一种将同质数据块（可以是连续或跨越的）解释为多维数组对象的方式。数据类型（dtype）决定了数据是怎样被解释为浮点数、整型、布尔型或其他任何类型。

ndarray这么灵活的一个原因是每个数组对象是一个数据块的跨度视图（strided view）。为什么数组视图`arr[::2, ::-1]没有复制数据呢？这是因为ndarray不只是一块内存和一个dtype，它还有跨度（striding）信息，这使得数组能以各种步幅（step size）在内存中移动。更加准确地讲，ndarray内部由一下部分组成：

+ A *pointer* to data --that is, a block of data in RAM or in a memory-mapped file
+ The *data type* or *dtype*, describing fixed-size value celss in the array
+ A tuple indicating the array's shape
+ A tuple of strides, integers indicating the number of types to 'step' in order to advance one element along a dimension一个跨度元组，其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数。

下图简单地说明了ndarray内部结构：
![ndarray内部结构](https://raw.githubusercontent.com/libingallin/python-for-data-anlysis/master/numpy/figs/numpy-ndarry-object.png)